In [38]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',
       'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
       'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
       'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
       'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
       'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS',
       'FEES', 'TRNFP', 'CBOM', 'VKCO', 'ENPG',
       'SGZH', 'FIVE', 'POLY', 'MOEX', 'GLTR',
       'HYDR', 'FIXP']

from TSMixer.TSMixer_models import TSMixer, all_data_scaller, sample_creator, winrate

In [40]:
path = '~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_(NEW)hourly_with_time_df'
total_df = pd.read_csv(path, index_col='tradedate')

In [41]:
total_df

,close,open,pr_change,spread_bbo,spread_lv10,spread_1mio,levels_b,levels_s,vol_b_x,vol_s_x,...,cancel_orders_s,cancel_val_b,cancel_val_s,cancel_vol_b,cancel_vol_s,cancel_vwap_b,cancel_vwap_s,cancel_vol,cancel_orders,secid
tradedate,,,,,,,,,,,,,,,,,,,,,
2020-01-03,259.10,259.43,-0.127202,1.625000,12.416667,4.483333,712.750000,628.000000,1754141.0,2687679.0,...,11538.0,2.019683e+10,6.651139e+09,7764608.0,2553676.0,263.020833,3119.01,10318284.0,33852.0,GAZP
2020-01-03,259.36,261.65,-0.875215,2.050000,12.433333,6.000000,854.916667,614.333333,2768711.0,2919966.0,...,18695.0,4.834394e+10,9.413088e+09,18535739.0,3596566.0,261.727500,3142.24,22132305.0,72679.0,GAZP
2020-01-03,261.80,261.33,0.179849,1.591667,11.416667,3.358333,937.583333,634.500000,3182151.0,3585278.0,...,8160.0,2.140887e+10,3.742319e+09,8144238.0,1428856.0,264.391667,3141.83,9573094.0,30026.0,GAZP
2020-01-03,261.54,261.20,0.130168,1.625000,11.008333,2.616667,946.166667,663.750000,3209152.0,4073639.0,...,6380.0,9.965698e+09,3.084672e+09,3806892.0,1107989.0,262.582500,3347.10,4914881.0,18429.0,GAZP
2020-01-03,261.20,259.54,0.639593,1.500000,10.666667,3.208333,896.666667,738.083333,3145721.0,4338093.0,...,6477.0,9.139586e+09,2.933773e+09,3503489.0,1107598.0,261.086667,3231.74,4611087.0,17535.0,GAZP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-27,362.90,363.50,-0.165062,6.250000,56.883333,26.875000,270.750000,451.000000,837875.0,2400243.0,...,1595.0,8.732842e+07,1.031099e+08,241577.0,282468.0,361.558333,4381.80,524045.0,3851.0,FIXP
2023-10-27,363.70,363.80,-0.027488,4.775000,54.758333,16.350000,273.416667,448.416667,796376.0,2445684.0,...,1648.0,1.049528e+08,1.001204e+08,290070.0,273939.0,361.875000,4386.90,564009.0,3656.0,FIXP
2023-10-27,363.70,363.20,0.137665,4.566667,56.966667,23.683333,266.500000,454.000000,389346.0,1248884.0,...,1048.0,8.848737e+07,8.859126e+07,245679.0,242997.0,360.450000,2187.80,488676.0,2617.0,FIXP


In [42]:
val_dates[0]

'2022-12-13'

In [24]:
result_df = pd.DataFrame(columns=(['Asset','Pred','Fact']))
winrate_dict = {}

for company in companies:
    data = total_df[total_df.secid == company].drop('secid', 1)
    train_data = data[ : round(data.shape[0]*0.80) ].to_numpy()
    val_data = data[ round(data.shape[0]*0.80) : ]
    val_dates = val_data.index.to_numpy()
    val_data = val_data.to_numpy()
    
    train_data, val_data, scaller_star = all_data_scaller(train_data, val_data)
    
    train_data1 = data[ : '2022-02-01' ].to_numpy()
    train_data2 = data[ '2022-05-01' : val_dates[0] ].to_numpy()
#=================================================================================
    lookback = 34
    horizon = 1
    x_train1, y_train1 = sample_creator(train_data1, lookback, horizon)
    x_train2, y_train2 = sample_creator(train_data2, lookback, horizon)
    x_val, y_val = sample_creator(val_data, lookback, horizon)
    x_train= torch.Tensor( np.concatenate([x_train1, x_train2]) )
    y_train = torch.Tensor( np.concatenate([y_train1, y_train2]) )
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset  = TensorDataset(x_val, y_val)                          
    train_loader = DataLoader(train_dataset, batch_size=150, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=150, shuffle=False)
    
#=================================================================================

    model = TSMixer(lookback, 57, horizon, 0.1, 1).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=3, verbose=True)
    
    epochs = 50
    early_stop_count = 0
    min_val_loss = float('inf')
    
    train_loss_per_epoch = []
    val_loss_per_epoch = []
    train_winrate = []
    val_winrate = []
    
    
    
    pbar = trange(epochs, desc="Epoch:")
    since = time.time()
    
    for epoch in pbar:
        model.train()
        train_losses = 0
        train_win = 0
        for batch in tqdm(train_loader, leave=False, desc="Train iter:"):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            train_losses += loss.item()
            optimizer.step()
            train_loss = train_losses/len(train_loader)
            train_win += winrate(y_batch.reshape(-1,).cpu().detach(), outputs.reshape(-1,).cpu().detach())
        train_loss_per_epoch.append(train_loss)    
        train_winrate.append(train_win / len(train_loader))
        
        # Validation
        model.eval()
        val_losses = 0
        val_win = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, leave=False, desc="Val iter:"):
                x_batch, y_batch = batch
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_losses += loss.item()
                val_win += winrate(y_batch.reshape(-1,).cpu().detach(), outputs.reshape(-1,).cpu().detach())
        val_loss = val_losses/len(val_loader)
        val_loss_per_epoch.append(val_loss)
        val_winrate.append(val_win/len(val_loader))
        
        scheduler.step(val_loss)
        #if val_loss < 0.0004:
            #descr = f'ВЕСА_БОТА/[34,57,1,0.1,1]|{company}|VW={val_win/len(val_loader)},VL={val_loss},TL={train_loss}'
            #torch.save(model.state_dict(), descr )
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            early_stop_count = 0
        else:
            early_stop_count += 1
    
        if early_stop_count >= 10:
            #descr = f'ВЕСА_БОТА/[34,57,1,0.1,1]|{company}|VW={val_win/len(val_loader)},VL={val_loss},TL={train_loss}'
            #torch.save(model.state_dict(), descr)
            break
#=================================================================================
    model.eval()
    predictions1 = []
    with torch.no_grad():
        for batch in val_loader:
            x_batch, y_batch = batch
            x_batch = x_batch.to(device)
            outputs = model(x_batch)
            predictions1.extend(outputs.squeeze().tolist())
    predictions1 = np.array(predictions1)  
    predictions1 = scaller_star.inverse_transform(predictions1.reshape(-1,1))
    y_val = scaller_star.inverse_transform(y_val)
    df = pd.DataFrame({'Asset':company,
          'Pred':predictions1.reshape(-1,),
          'Fact':y_val.reshape(-1,)})
    df.index = val_dates[lookback:]
    result_df = pd.concat([result_df,df])
    
    descr = f'ВЕСА_БОТА/[34,57,1,0.1,1]|{company}|VW={val_win/len(val_loader)},VL={val_loss},TL={train_loss}'
    torch.save(model.state_dict(), descr )
    winrate_dict[company] = val_win/len(val_loader)

/tmp/ipykernel_8561/2170866909.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = total_df[total_df.secid == company].drop('secid', 1)


Epoch::   0%|          | 0/50 [00:00<?, ?it/s]

Train iter::   0%|          | 0/43 [00:00<?, ?it/s]

Val iter::   0%|          | 0/12 [00:00<?, ?it/s]

Train iter::   0%|          | 0/43 [00:00<?, ?it/s]

...

Epoch 00028: reducing learning rate of group 0 to 2.4000e-05.


Train iter::   0%|          | 0/29 [00:00<?, ?it/s]

Val iter::   0%|          | 0/8 [00:00<?, ?it/s]

Train iter::   0%|          | 0/29 [00:00<?, ?it/s]

Val iter::   0%|          | 0/8 [00:00<?, ?it/s]

In [36]:
winrate_dict

{'GAZP': 0.6213646532438478,
 'SBER': 0.6644209969408261,
 'LKOH': 0.6680902989627822,
 'GMKN': 0.750559284116331,
 'MGNT': 0.7736424649176328,
 'TATN': 0.6942527649116174,
 'NVTK': 0.6051175541384238,
 'SNGS': 0.7720851448734304,
 'PLZL': 0.6497070549198893,
 'SNGSP': 0.6351348468493069,
 'PIKK': 0.5067114093959731,
 'ROSN': 0.7511560876059833,
 'SBERP': 0.6342687209069149,
 'CHMF': 0.7108501118568232,
 'NLMK': 0.638355891471903,
 'IRAO': 0.720917225950783,
 'YNDX': 0.5716286031577422,
 'ALRS': 0.62751677852349,
 'RUAL': 0.5859100484094362,
 'MTSS': 0.7043742451220879,
 'MAGN': 0.5417937766931056,
 'PHOR': 0.5329594557322792,
 'RTKM': 0.5166641244661379,
 'VTBR': 0.5887605682907696,
 'TATNP': 0.5717759884505322,
 'TCSG': 0.6390705428421658,
 'AGRO': 0.4951901565995525,
 'OZON': 0.5476700461706888,
 'AFLT': 0.4935123042505592,
 'AFKS': 0.5690156599552573,
 'FEES': 0.5055560092816341,
 'TRNFP': 0.5308980134055102,
 'CBOM': 0.686241610738255,
 'VKCO': 0.5379271479071674,
 'ENPG': 0.48371

In [10]:
result_df['date'] = result_df.index

res = [1]
h = 2
for i in range(1, 66297):
    if result_df.iloc[i,3] == result_df.iloc[i-1,3]:
        res.append(h)
        h+=1
    if result_df.iloc[i,3] != result_df.iloc[i-1,3]:
        h=1
        res.append(h)
        h+=1
result_df['hour'] = res

In [14]:
pct_pred = np.array([])
pct_fact = np.array([])
for company in companies:
    pred_change = np.array( result_df[result_df.Asset == company].Pred.pct_change().fillna(0) )
    fact_change = np.array( result_df[result_df.Asset == company].Fact.pct_change().fillna(0) )
    pct_pred = np.concatenate([pct_pred, pred_change])
    pct_fact = np.concatenate([pct_fact, fact_change])

In [16]:
result_df['pct_pred'] = pct_pred
result_df['pct_fact'] = pct_fact

In [23]:
result_df.to_csv('backtest_data')